In [ ]:
import pandas as pd
import numpy as np

Необходимо привести данные к виду:
|Имя дисциплины|Посещаемость лекций 1 пром|Посещаемость лекций 2 пром|Посещаемость лекций 3 пром|Посещаемость практик 1 пром|Посещаемость практик 2 пром|Посещаемость практик 3 пром|КТ 1|КТ 2|Экзамен|Балл ктр|Балл лаб|
|-|-|-|-|-|-|-|-|-|-|-|-|
|Имя-семестр|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|1|1|

# Реализация

Берем таблицу attendance-students

In [ ]:
attendance_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-students.csv').rename(columns={'id':'attendance_id', 'contingent_id':'con_id'})

attendance_students.head(5)

Берем таблицу ocenka-students

In [ ]:
ocenka_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-students.csv').rename(columns={'id':'ocenka_id', 'study_id':'con_id'})

ocenka_students.head(5)

Добавляем эти таблицы в мастер-таблицу

In [ ]:
master_table = attendance_students.set_index('con_id').join(ocenka_students.set_index('con_id'))

master_table.head(5)

Удаляем из таблицы все кроме id

In [ ]:
master_table.drop(columns=['enrollment_at', 'deleted_at'], axis=1, inplace=True)

master_table.head(5)

Добавляем в таблицу группы и удаляем все, кроме id студентов и групп.

В дальнейшем в принципе будет удаляться все, кроме интересующей информации, даже если придется её потом снова добавлять

In [ ]:
ocenka_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-groups.csv').rename(columns={'id':'group_id'})

ocenka_groups.head(5)

In [ ]:
master_table = master_table.join(ocenka_groups.rename(columns={'number':'group_num'}).set_index('group_id'), on='group_id')

master_table.drop(['group_id', 'edu_program_full_title'], axis=1, inplace=True)

master_table.head(5)

Получим id групп со всех баз

In [ ]:
attendance_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-groups.csv').rename(columns={'id':'group_id'})

attendance_groups.head(5)

In [ ]:
timetable_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-groups.csv').rename(columns ={'id':'group_id'})

timetable_groups.head(5)

In [ ]:
master_table = master_table.join(ocenka_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_ocenka'}).set_index('group_num'), on='group_num')
master_table = master_table.join(attendance_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_attendance'}).set_index('group_num'), on='group_num')
master_table = master_table.join(timetable_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_timetable'}).set_index('group_num'), on='group_num')

master_table = master_table.dropna()
master_table = master_table.reset_index()

master_table.head(10)

Добавляем данные о семестре

In [ ]:
timetable_courses = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-courses.csv').rename(columns={'id':'course_id'})

timetable_courses.head(5)

In [ ]:
timetable_faculties = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-faculties.csv').rename(columns={'id':'faculty_id'})

timetable_faculties.head(5)

In [ ]:
bufer_table = timetable_courses[['course_id', 'faculty_id']]

bufer_table.head(5)

In [ ]:
bufer_table = bufer_table.join(timetable_faculties[['faculty_id', 'semester_id']].set_index('faculty_id'), on='faculty_id')

bufer_table.head(5)

In [ ]:
bufer_table = bufer_table.set_index(['course_id', 'faculty_id']).sub(bufer_table['semester_id'].min()).reset_index()[['course_id', 'semester_id']]

bufer_table.head(10)

In [ ]:
master_table = master_table.join(timetable_groups[['group_id', 'course_id']].rename(columns={'group_id':'group_id_timetable'}).set_index('group_id_timetable'), on='group_id_timetable')

master_table.head(10)

In [ ]:
master_table = master_table.join(bufer_table.set_index('course_id'), on='course_id')

master_table.head(5)

In [ ]:
master_table.drop(['course_id'], axis=1,inplace=True)
master_table = master_table.rename(columns={'semester_id':'semester'})

master_table.head(5)

Находим все дисциплины группы

In [ ]:
timetable_educations = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-educations.csv').rename(columns={'id':'edu_id'})

timetable_educations.head(5)

In [ ]:
timetable_disciplines = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-disciplines.csv').rename(columns={'id':'discipline_id'})

timetable_disciplines.head(5)

In [ ]:
bufer_table = timetable_educations

bufer_table = bufer_table.join(timetable_disciplines[['discipline_id', 'name']].rename(columns={'name':'names'}).set_index('discipline_id'), on='discipline_id').dropna()

bufer_table.head(5)

In [ ]:
master_table = master_table.join(bufer_table[['group_id', 'discipline_id', 'names']].rename(columns={'discipline_id':'discipline_id_timetable', 'names':'dicipline_name_timetable'}).set_index('group_id'), on='group_id_timetable').dropna().reset_index()

master_table.drop(['index'], axis=1,inplace=True)

master_table.head(5)

Считаем посещаемость

In [ ]:
attendance_lessons = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-lessons.csv")

attendance_lessons.head(5)

In [ ]:
attendance_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-disciplines.csv")

attendance_disciplines.head(5)

In [ ]:
attendance_presences = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-presences.csv")

attendance_presences['state'].fillna('wasnt', inplace=True)

attendance_presences.head(5)

In [ ]:
bufer_table = attendance_lessons[['id', 'group_id', 'discipline_id', 'date_on', 'kind']].loc[((attendance_lessons['kind'] == 'lecture') |(attendance_lessons['kind'] == 'practice'))]
bufer_table.head(5)

In [ ]:
bufer_table = bufer_table.join(attendance_presences[['student_id', 'lesson_id', 'state']].set_index('lesson_id'), on='id').rename(columns={'date_on':'date'}).dropna().reset_index()

bufer_table.drop(['index', 'id'], axis=1, inplace=True)

bufer_table.head()

In [ ]:
counter = 0
for i in range(2018,2030):
    bufer_table.loc[(bufer_table['date'] >= f'{i}-09-01') & (bufer_table['date'] < f'{i}-10-16'), 'period'] = 1
    bufer_table.loc[(bufer_table['date'] >= f'{i}-10-16') & (bufer_table['date'] < f'{i}-12-04'), 'period'] = 2
    bufer_table.loc[(bufer_table['date'] >= f'{i}-12-04') & (bufer_table['date'] < f'{i+1}-01-15'), 'period'] = 3
    
    bufer_table.loc[(bufer_table['date'] >= f'{i}-09-01') & (bufer_table['date'] <= f'{i+1}-01-15'), 'semester'] = counter
    counter+=1
    
    
    bufer_table.loc[(bufer_table['date'] >= f'{i+1}-02-03') & (bufer_table['date'] < f'{i+1}-03-15'), 'period'] = 1
    bufer_table.loc[(bufer_table['date'] >= f'{i+1}-03-15') & (bufer_table['date'] < f'{i+1}-04-15'), 'period'] = 2
    bufer_table.loc[(bufer_table['date'] >= f'{i+1}-04-15') & (bufer_table['date'] <= f'{i+1}-06-30'), 'period'] = 3
    
    bufer_table.loc[(bufer_table['date'] >= f'{i+1}-02-03') & (bufer_table['date'] <= f'{i+1}-06-30'), 'semester'] = counter
    counter+=1

bufer_table.dropna(inplace=True)

bufer_table.drop('date', axis=1, inplace=True)

bufer_table.head(5)

In [ ]:
bufer_table.loc[bufer_table['state'] == 'was', 'state'] = 1
bufer_table.loc[bufer_table['state'] == 'wasnt', 'state'] = 0

bufer_table = bufer_table.reset_index()

bufer_table.drop('index', axis=1, inplace=True)

bufer_table.head(5)

In [ ]:
sum = bufer_table
sum = sum.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].sum().reset_index().rename(columns={'state':'count_presences'})

sum.head(5)

In [ ]:
count = bufer_table
count = count.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].count().reset_index().rename(columns={'state':'count_lessons'})

count.head(5)

In [ ]:
bufer_table = sum.join(count[['count_lessons']])

bufer_table.head(5)

In [ ]:
bufer_table['attendance'] = bufer_table['count_presences'] / bufer_table['count_lessons']

bufer_table.drop('count_presences', axis=1, inplace=True)
bufer_table.drop('count_lessons', axis=1, inplace=True)

bufer_table.head(5)

In [ ]:
bufer_table = bufer_table.join(attendance_disciplines.set_index('id'), on='discipline_id')

bufer_table.drop('discipline_id', axis=1, inplace=True)
bufer_table.drop('group_id', axis=1, inplace=True)
bufer_table = bufer_table.rename(columns={'student_id':'attendance_id'})

bufer_table.head(5)

In [ ]:
master_table = master_table.join(bufer_table.set_index(['attendance_id', 'semester']), on=['attendance_id', 'semester']).dropna()

master_table.head(5)

Собираем оценки

In [ ]:
ocenka_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-semesters.csv").rename(columns={'id':'semester_id'})

ocenka_semesters.head()

In [ ]:
ocenka_discipline_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semesters.csv").rename(columns={'id':'discipline_semester_id'})

ocenka_discipline_semesters.head(5)

In [ ]:
ocenka_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-disciplines.csv").rename(columns={'id':'discipline_id'})

ocenka_disciplines.head(5)

In [ ]:
ocenka_marks = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-marks.csv").rename(columns={'id':'mark_id'})

ocenka_marks.head(5)

In [ ]:
ocenka_discipline_semesters_periods = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semester-periods.csv").rename(columns={'id':'discipline_semester_period_id'})